In [1789]:
from google import genai
from dotenv import load_dotenv
import os
from google.genai.types import GenerateContentConfig, Tool
import json
import re
import random as rd

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the .env file
api_key = os.getenv('GEMINI_API_KEY')

# Initialize the Google AI client
client = genai.Client(api_key=api_key)

# Generate content
response = client.models.generate_content(
    model='gemini-2.0-flash-exp', contents='What is your name?'
)
print(response.text)




I am a large language model, trained by Google.



*Function to extract Json of LLm output and save it to a file*

In [1790]:
def save_json_from_string(input_string, output_filename):
    """
    Extracts JSON content from a string and saves it to a JSON file with the specified name.

    :param input_string: The input string containing JSON data within brackets.
    :param output_filename: The name of the output JSON file (e.g., "data.json").
    :return: True if the operation was successful, False otherwise.
    """
    # Step 1: Extract the JSON content within the brackets
    result = re.search(r'\[.*\]', input_string, re.DOTALL)

    if result:
        extracted_content = result.group(0)  # Get the matched content

        try:
            # Step 2: Parse the extracted content into a Python object (list/dict)
            json_data = json.loads(extracted_content)

            # Step 3: Write the JSON data to a file
            with open(output_filename, "w", encoding="utf-8") as json_file:
                json.dump(json_data, json_file, indent=4)  # Save with pretty formatting

            print(f"JSON data has been successfully written to '{output_filename}'.")
            return True  # Operation was successful
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")
        except Exception as e:
            print(f"An error occurred while writing the file: {e}")
    else:
        print("No content found within brackets.")

    return False  # Operation failed

In [1791]:
MODEL = "gemini-2.0-flash-exp"

In [1792]:

COMPANY = 'Proximus'

In [1793]:
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

**1. SETTING UP A RESEARCH ASSISTANT** (temperature = 0, tools = google search)

In [1794]:
system_instruction = """You are an analyst that conducts company research.
You are given a company name, and you will work on a company report. You have access
to Google Search to look up company news, updates, metrics, public records and linkedin pages to write research reports.

When given a company name, identify key aspects to research, look up that information
and then write an elaborate company report. 

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report."""

In [1795]:
config = GenerateContentConfig(system_instruction=system_instruction, tools=[Tool(google_search={})], temperature=0,maxOutputTokens=8000, top_p=0.9, top_k=5, safety_settings=safety_settings)

**Proximus News report**

In [1796]:
contents = f"""
Write a report about {COMPANY}.

The report should contain an extensive overview of the most important news facts of the last 2 weeks. 

The report should be written in this style example:

Use this JSON schema:

Proximus_news = {{'subject':str, 'overview':str, 'source':str}}
Return: list[Proximus_news]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)



In [1797]:
save_json_from_string(response.text, 'proximus_news.json')

JSON data has been successfully written to 'proximus_news.json'.


True

**Proximus Key Employees and Roles**

In [1798]:
contents = f"""
Write a report about {COMPANY}

The report should only contain a comprehensive summary of the employees in the company. 

The report should contain, the complete first name and family name, an email address with this structure "first name.family name@{COMPANY}.be" (put it all in lower case), the role that they have within the company and the department that they make part of.

Do not include abbreviations in names, and exclude employees whose full first and last names cannot be found.

Give me as much grounded names that you can find with your research and put them all in the report.

The report should be written in this style example:

Use this JSON schema:

Proximus_employees = {{'first_name':str, 'family_name':str, 'role':str, 'department':str, 'email_address':str}}
Return: list[Proximus_employees]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)


In [1799]:
save_json_from_string(response.text, 'proximus_employees.json')

JSON data has been successfully written to 'proximus_employees.json'.


True

**Proximus Departments**

In [1800]:
contents = f"""

Write a report about {COMPANY}

The report should exclusively provide a detailed summary of the company's departments and their respective subdivisions, if any.

Include in the report the full address where the headquarter of the provider is located, the main phone number and vat number.

Do not include abbreviations in the report.

Provide a detailed list, specifying only verified and public information. Do not include speculative or incomplete details.

The report should be written in this style example:

Use this JSON schema:

Proximus_departments = {{'department':str, 'subdivision':list[str], 'address':str, 'phone':str, 'vat':str}}
Return: list[Proximus_departments]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)


In [1801]:
save_json_from_string(response.text, 'proximus_departments.json')

JSON data has been successfully written to 'proximus_departments.json'.


True

**Proximus Colors**

In [1802]:
contents = f"""

Write a report about {COMPANY}

The report should exclusively provide the main colors of Company's branding, including any official color codes such as HEX or RGB. Focus on the primary colors used in the company's logo and website.

Do not include speculative colors and focus only on the main ones.

The report should be written in this style example:

Use this JSON schema:

company_colors = {{'color_name_1':str, 'hex_code_1':str, 'rgb_code_1':list[int]}},{{'color_name_2':str,'hex_code_2':str,'rgb_code_2':list[int]}}
Return: list[company_colors]

Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
"""


response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)



In [1803]:
save_json_from_string(response.text, 'proximus_colors.json')

JSON data has been successfully written to 'proximus_colors.json'.


True

**Proximus Service providers**

In [1804]:
contents = f"""

Write a report about {COMPANY}

List the main service providers of the company. Include any known suppliers, contractors, technology service providers, or any other third-party companies that the company relies on. 
Provide the names of the service providers, type and description of the services they provide to the company. Also include the providers homepage in the report, if you can't find it, don not include the provider in the report. Don t include providers with brackets or special characters in their company name. 

The report should be written in this style example:

Use this JSON schema:

company_service = {{"provider":str,"service":str,"type":str,"provider_homepage":str]}}
Return: list[company_service]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)



In [1805]:
save_json_from_string(response.text, 'proximus_providers.json')

JSON data has been successfully written to 'proximus_providers.json'.


True

**Merge the proximus data**

In [1806]:

# File paths for input and output
input_files = {
    "colors": "proximus_colors.json",
    "departments": "proximus_departments.json",
    "employees": "proximus_employees.json",
    "news": "proximus_news.json",
    "providers": "proximus_providers.json"
}
output_file = "blended_data_proximus.json"

# Combine all data into one dictionary
blended_data = {}

for key, file_path in input_files.items():
    with open(file_path, 'r') as f:
        blended_data[key] = json.load(f)

# Write the combined data to a new JSON file
with open(output_file, 'w') as f:
    json.dump(blended_data, f, indent=4)

print(f"Data successfully merged into {output_file}")

Data successfully merged into blended_data_proximus.json


**Providers info** (Proceed with one Provider)

In [1807]:
# Load JSON data from the file
with open("proximus_providers.json", "r") as file:
    providers = json.load(file)

In [1808]:
# Function to pick a random provider
def pick_random_provider(providers):
    if not providers:
        return None  # Handle the case where the list is empty
    return rd.choice(providers)

In [1809]:
provider = pick_random_provider(providers)

print(provider)

{'provider': 'Comviva', 'service': 'Technology platform', 'type': "Provides the BlueMarble platform for the 'Tech Hub' platform being built by Tech Mahindra.", 'provider_homepage': None}


In [1810]:
provider = [provider]
file_path = 'provider_general_data.json'

# Write the data to the JSON file
with open(file_path, 'w') as json_file:
    json.dump(provider, json_file, indent=4)

print(f'Data saved to {file_path}')

Data saved to provider_general_data.json


**Provider Key Employees and Roles**

In [1811]:
contents = f"""
    Write a report about company {provider}. This company is a provider of {COMPANY}

    The report should only contain a comprehensive summary of the employees in the company.

    The report should contain, the complete first name and family name, an email address with this structure "first name.family name@{provider}.be" (put it all in lower case), the role that they have within the company and the department that they make part of.

    Do not include abbreviations in names, and exclude employees whose full first and last names cannot be found. Make sure the first and the last name contains more then 2 characters. Do not include employees of {COMPANY} in the report.

    Give me as much grounded names that you can find with your research and put them all in the report.

    The report should be written in this style example:

    Use this JSON schema:

    Employees = {{'provider': str, 'first_name': str, 'family_name': str, 'role': str, 'department': str, 'email_address':str}}
    Return: list[Employees]
    """
    
response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)


    

In [1812]:
save_json_from_string(response.text, 'provider_employees.json')

JSON data has been successfully written to 'provider_employees.json'.


True

**Provider Colors**

In [1813]:

contents = f"""

    Write a report about company {provider}. This company is a provider of {COMPANY}

    The report should exclusively provide the main colors of Company's branding, including any official color codes such as HEX or RGB. Focus on the primary colors used in the company's logo and website.

    Do not include speculative colors and focus only on the main ones.

    The report should be written in this style example:

    Use this JSON schema:

    company_colors = {{'provider':str,'color_name_1':str, 'hex_code_1':str, 'rgb_code_1':list[int]}},{{'color_name_2':str,'hex_code_2':str,'rgb_code_2':list[int]}}
    Return: list[company_colors]

    Thoroughly plan your work in detail and steps, but avoid discussing it. Do not add any additional comments after finishing the report.
    """


response = client.models.generate_content(
        model=MODEL, config=config, contents=contents)



In [1814]:
save_json_from_string(response.text, 'provider_color.json')

JSON data has been successfully written to 'provider_color.json'.


True

**Provider Departments**

In [1815]:
contents = f"""

Write a report about company {provider}. This company is a provider of {COMPANY}

The report should exclusively provide a detailed summary of the company's departments and their respective subdivisions, if any.

Include in the report the full address where the headquarter of the provider is located, the main phone number and vat number.

Do not include abbreviations in the report.

Provide a detailed list, specifying only verified and public information. Do not include speculative or incomplete details.

The report should be written in this style example:

Use this JSON schema:

Provider_departments = {{'provider':str,'department':str, 'subdivision':list[str],'address':str, 'phone':str, 'vat':str}}
Return: list[Provider_departments]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)



In [1816]:
save_json_from_string(response.text, 'provider_departments.json')

JSON data has been successfully written to 'provider_departments.json'.


True

**Blend Data from Json files Providers**

In [1817]:
# Function to read JSON data from a file and debug if necessary
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
            # Debugging: Print the entire content of the file to inspect the structure
            print(f"Data loaded from {file_path} (Type: {type(data)}): {data}")
            if not isinstance(data, list):
                raise ValueError(f"Data in {file_path} is not a list as expected.")
            return data
        except json.JSONDecodeError:
            print(f"Error decoding JSON from {file_path}")
            return []  # Return an empty list if the JSON is malformed

# Function to blend data
def blend_data_from_files(general_file, employee_file, color_file, department_file):
    # Read data from files
    general_data = read_json_file(general_file)
    employee_data = read_json_file(employee_file)
    color_data = read_json_file(color_file)
    department_data = read_json_file(department_file)
    
    # Merge data
    merged_data = {}

    # Add general data
    for entry in general_data:
        try:
            provider = entry["provider"]
            merged_data[provider] = {"general": entry}
        except KeyError:
            print("KeyError: Missing 'provider' in general data entry.")
            continue

    # Add employee data
    for entry in employee_data:
        try:
            provider = entry["provider"]
            if provider in merged_data:
                if "employee" not in merged_data[provider]:
                    merged_data[provider]["employee"] = []
                merged_data[provider]["employee"].append(entry)
        except KeyError:
            print("KeyError: Missing 'provider' in employee data entry.")
            continue

    # Add color data
    for entry in color_data:
        try:
            provider = entry.get("provider", "Unknown")
            if provider in merged_data:
                if "color" not in merged_data[provider]:
                    merged_data[provider]["color"] = {}
                merged_data[provider]["color"].update(entry)
        except KeyError:
            print("KeyError: Missing 'provider' in color data entry.")
            continue

    # Add department data
    for entry in department_data:
        try:
            provider = entry["provider"]
            if provider in merged_data:
                if "departments" not in merged_data[provider]:
                    merged_data[provider]["departments"] = []
                merged_data[provider]["departments"].append(entry)
        except KeyError:
            print("KeyError: Missing 'provider' in department data entry.")
            continue

    return merged_data

# File paths
general_file = 'provider_general_data.json'
employee_file = 'provider_employees.json'
color_file = 'provider_color.json'
department_file = 'provider_departments.json'

# Debugging: Print current working directory
print("Current working directory:", os.getcwd())

# Blend the data
blended_result = blend_data_from_files(general_file, employee_file, color_file, department_file)

# Debugging: Print the blended data to verify
print("Blended Data:", json.dumps(blended_result, indent=4))

# Write the result to a new JSON file and capture any potential errors
try:
    with open('blended_data_providers.json', 'w') as output_file:
        json.dump(blended_result, output_file, indent=4)
    print("Blended data has been saved to 'blended_data_providers.json'")
except Exception as e:
    print(f"Error saving file: {e}")






Current working directory: c:\Users\Rik\Desktop\Proximus-Case-Team-4\Rik
Data loaded from provider_general_data.json (Type: <class 'list'>): [{'provider': 'Comviva', 'service': 'Technology platform', 'type': "Provides the BlueMarble platform for the 'Tech Hub' platform being built by Tech Mahindra.", 'provider_homepage': None}]
Data loaded from provider_employees.json (Type: <class 'list'>): [{'provider': 'Comviva', 'first_name': 'Rajesh', 'family_name': 'Chandiramani', 'role': 'Chief Executive Officer', 'department': 'Executive Management', 'email_address': 'rajesh.chandiramani@comviva.be'}, {'provider': 'Comviva', 'first_name': 'Rishi', 'family_name': 'Partab Ramphal', 'role': 'Chief Delivery Officer', 'department': 'Executive Management', 'email_address': 'rishi.partabramphal@comviva.be'}, {'provider': 'Comviva', 'first_name': 'Amit', 'family_name': 'Sanyal', 'role': 'Chief Operating Officer, MarTech', 'department': 'Executive Management', 'email_address': 'amit.sanyal@comviva.be'},

**2. SETTING UP THE EMAIL WRITING ASSISTANT** (temperature = 1, tools= ??)

In [1818]:
email_tones = [
    "Urgent",
    "Firm",
    "Action Required",
    "Time-Sensitive",
    "Critical",
    "Pressing",
    "Immediate Response",
    "High-Priority",
    "Polite",
    "Friendly",
    "Professional",
    "Supportive",
    "Respectful",
    "Casual",
    "Empathetic"
]

In [1819]:
random_tone = rd.choice(email_tones)

In [1820]:
print(random_tone)

Professional


In [1821]:
system_instruction= f"""You are a skilled copywriter with a knack for creating emails that feel
    personal, relevant, urgent and engaging. Your task is to write emails to employees of
    {COMPANY} all while maintaining a tone that feels individual, tailored and professional. 
    
    The email should appear to come from a relevant source. Encourage actions like clicking links or downloading attachments with a sense of urgency. Align the email subject with the sender's theme and message.
    
    Incorporate in your answer only complete emails.
    
    Everything about the sender address, subject, and email body is focused on the recipient interacting with the link or attachment.
    
    The email should have a {random_tone} tone.
    """

In [1822]:
config = GenerateContentConfig(system_instruction=system_instruction, temperature=1,maxOutputTokens=8000, top_p=0.9, top_k=40, safety_settings=safety_settings)

**Fetch some Provider Context to generate better emails**

In [1823]:
# Load JSON data from the file
with open("provider_employees.json", "r") as file:
    provider_employees = json.load(file)

In [1824]:
# Load JSON data from the file
with open("provider_departments.json", "r") as file:
    provider_departments = json.load(file)

**Setting up some dummy data about proximus employees**

In [1825]:
proximus_employees = [
    {
        "name": "Marie Declercq", 
        "email": "marie.declercq@proximus.be", 
        "role": "Data Analyst", 
        "language": "Dutch", 
        "department": "Consumer Market"
    },
    {
        "name": "Luc Vandenberg", 
        "email": "luc.vandenberg@proximus.be", 
        "role": "Cybersecurity Specialist", 
        "language": "Dutch", 
        "department": "Network & Wholesale"
    },
    {
        "name": "Emma Wouters", 
        "email": "emma.wouters@proximus.be", 
        "role": "Marketing Specialist", 
        "language": "English", 
        "department": "Enterprise Market"
    },
    {
        "name": "Jai Mehta", 
        "email": "jai.mehta@proximus.be", 
        "role": "System Administrator", 
        "language": "English", 
        "department": "IT Infrastructure"
    },
    {
        "name": "Guillaume Boutin", 
        "email": "guillaume.boutin@proximus.be", 
        "role": "Chief Executive Officer", 
        "language": "French", 
        "department": "Chief Executive Officer's Department"
    }
] 


In [1826]:
random_employee = rd.choice(proximus_employees)

In [1827]:
print(random_employee)

{'name': 'Emma Wouters', 'email': 'emma.wouters@proximus.be', 'role': 'Marketing Specialist', 'language': 'English', 'department': 'Enterprise Market'}


In [1828]:
telecom_employee_events = [
    "Seminars",
    "Presentations",
    "Networking Meetings",
    "Workshops",
    "Training Sessions",
    "Industry Roundtables",
    "Panel Discussions",
    "Webinars",
    "Conferences",
    "Hackathons",
    "Product Demos",
    "Leadership Forums",
    "Strategic Planning Meetings",
    "Customer Experience Sessions",
    "Innovation Labs"
]

In [1829]:
random_event=rd.choice(telecom_employee_events)

In [1830]:
print(random_event)

Panel Discussions


In [1831]:
urgent_provider_issues = [
    "Network Outages",
    "SLA Violations",
    "Payment Delays",
    "Contract Breach",
    "Quality Failures",
    "Security Risks",
    "Price Increases",
    "Missed Deadlines",
    "Regulatory Issues",
    "Billing Discrepancies"
]

In [1832]:
random_issue=rd.choice(urgent_provider_issues)

In [1833]:
print(random_issue)

Missed Deadlines


**Emails Provider to Proximus**

Service related Emails

In [1834]:
contents = f"""

{provider} is a service provider of {COMPANY}. {provider} has these {provider_departments}.

Write some tailored emails based on the role of {random_employee['role']} about an urgent matter that needs to be solved related to {COMPANY} concerning a service that {provider} offers. Elaborate about the issue and point out why it needs to be solved as quick as possible.

The receiver of the email is {random_employee['name']} and should be in {random_employee['language']}. Address the receiver formal and by his last name.

The email signature should contain the first name, family name, role and the company name of one of the employees of {provider}. 

Do not include the link in the body of the email, as it will be implemented separately and placed below the text body, refer to it in the text body where it is placed.

Just plain text in the body, nothing to include anymore.

The email should be written in this style example:

Use this JSON schema:

Provider_email = {{'subject':str,'body':str, 'email_sign_fullname':str, 'email_sign_role':str, 'email_sign_company':str, 'receiver_full_name':str}}
Return: list[Provider_email]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)



repairing a string

In [1835]:
def repair_json_response(response_text):
    """
    Repairs a potentially malformed JSON string from `response.text`.
    Ensures it starts with '[' and ends with ']' after the last complete dictionary.

    Args:
        response_text (str): The response text containing a potentially malformed JSON string.

    Returns:
        None: Prints the repaired JSON string.

    Raises:
        ValueError: If the string cannot be repaired into valid JSON.
    """
    # Remove the unwanted prefix if it exists
    if response_text.strip().startswith("```json") or response_text.strip().startswith("[```json"):
        response_text = response_text.replace("```json", "", 1).replace("[```json", "", 1).strip()

    # Ensure the string starts with '['
    if not response_text.strip().startswith("["):
        response_text = "[" + response_text

    # Find the last closing brace ('}')
    last_brace_index = response_text.rfind("}")
    if last_brace_index == -1:
        raise ValueError("No closing brace found in the response text.")

    # Trim the string up to the last complete dictionary and close the list
    repaired_string = response_text[:last_brace_index + 1].rstrip(", \n") + "]"
    
    # Print the repaired JSON string
    return repaired_string


In [1836]:
response_llm = response.text
repair_json_response(response_llm)

'[\n  {\n    "subject": "Urgent: BlueMarble Platform Integration Issue Impacting Customer Experience",\n    "body": "Dear Ms. Wouters,\\n\\nI hope this email finds you well.\\n\\nI am writing to you today regarding an urgent matter that requires your immediate attention. Our team has identified a critical integration issue with the BlueMarble platform, which is impacting the \'Tech Hub\' platform and, consequently, the customer experience. \\n\\nSpecifically, the issue is causing inconsistencies in the digital experience and management. This is leading to customer dissatisfaction and could potentially harm the Proximus brand if not addressed quickly. We need to rectify the situation as fast as possible, to make sure that no further impact is made on our customer experience. We have prepared a detailed report outlining the problem and proposed solutions. \\n\\nPlease review the attached document at your earliest convenience.\\n\\nThank you for your immediate attention to this matter.\\n

In [1837]:
save_json_from_string(response_llm, 'emails_related_service.json')

JSON data has been successfully written to 'emails_related_service.json'.


True

Provider Proximus Employee events

In [1838]:
contents = f"""

{provider} is a service provider of {COMPANY}. {provider} has these {provider_departments}.

Write some tailored emails based on the role of {random_employee['role']} about an event that is organized by {provider}. 

The event is {random_event} and is geared towards {random_employee['role']}.

Do not include urgent in the subject of the email. State that the spots available are limited and urgent action is required.

The receiver of the email is {random_employee['name']} and should be in {random_employee['language']}. Address the receiver formal and by his last name.

The email signature should contain the first name, family name, role and the company name of one of the employees of {provider}. 

Do not include the link in the body of the email, as it will be implemented separately and placed below the text body, refer to it in the text body where it is placed.

Just plain text in the body, nothing to include anymore.

The email should be written in this style example:

Use this JSON schema:

Provider_email = {{'subject':str,'body':str, 'email_sign_fullname':str, 'email_sign_role':str, 'email_sign_company':str, 'receiver_full_name':str}}
Return: list[Provider_email]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)



In [1839]:
response_llm = response.text
repair_json_response(response_llm)

'[\n  {\n    "subject": "Exclusive Panel Discussion for Marketing Specialists",\n    "body": "Dear Wouters,\\n\\nWe are excited to invite you to an exclusive panel discussion tailored specifically for Marketing Specialists. This event, hosted by Comviva, will delve into the latest trends and strategies in the marketing landscape, focusing on how technology platforms like BlueMarble are revolutionizing customer engagement and digital experiences.\\n\\nThis is a unique opportunity to connect with industry leaders and gain valuable insights to enhance your marketing strategies. Spaces are limited, so we encourage you to register promptly using the link below. \\n\\nWe look forward to seeing you there.",\n    "email_sign_fullname": "Priya Sharma",\n    "email_sign_role": "Marketing Director",\n    "email_sign_company": "Comviva",\n    "receiver_full_name": "Emma Wouters"\n  },\n  {\n    "subject": "Comviva Panel Discussion: Elevating Your Marketing Strategies",\n     "body": "Dear Wouters,

In [1840]:
save_json_from_string(response_llm, 'emails_employee_event.json')

JSON data has been successfully written to 'emails_employee_event.json'.


True

Provider issues with Proximus

In [1841]:
contents= f"""

{provider} is a service provider of {COMPANY}. {provider} has these {provider_departments}.

Write some tailored emails based on the role of {random_employee['role']} about an issue that {provider} have with {COMPANY}. 

The issue is {random_issue} and is related to {random_employee['role']}.

Do not include urgent in the subject of the email.

The receiver of the email is {random_employee['name']} and should be in {random_employee['language']}. Address the receiver formal and by his last name.

The email signature should contain the first name, family name, role and the company name of one of the employees of {provider}. 

Do not include the link in the body of the email, as it will be implemented separately and placed below the text body, refer to it in the text body where it is placed.

Just plain text in the body, nothing to include anymore.

The email should be written in this style example:

Use this JSON schema:

Provider_email = {{'subject':str,'body':str, 'email_sign_fullname':str, 'email_sign_role':str, 'email_sign_company':str, 'receiver_full_name':str}}
Return: list[Provider_email]
"""

response = client.models.generate_content(
    model=MODEL, config=config, contents=contents)



In [1842]:
response_llm = response.text
repair_json_response(response_llm)

'[\n  {\n    "subject": "Project Timeline Update - BlueMarble Platform",\n    "body": "Dear Ms. Wouters,\\n\\nI hope this email finds you well.\\n\\nI am writing to you today regarding the recent deadlines for the BlueMarble platform implementation. We have noted some discrepancies in the timeline adherence, which directly impacts our joint marketing strategies. To address this and ensure we are aligned on the path forward, please review the attached document detailing the missed deadlines and proposed adjustments. Your prompt attention to this matter is highly appreciated.\\n\\nThank you for your understanding.\\n\\n",\n    "email_sign_fullname": "Priya Sharma",\n    "email_sign_role": "Project Manager",\n    "email_sign_company": "Comviva",\n     "receiver_full_name": "Emma Wouters"\n  },\n  {\n    "subject": "Action Required: Review of Marketing Timeline for BlueMarble",\n    "body": "Dear Ms. Wouters,\\n\\nI trust you are having a productive week.\\n\\nThis email pertains to the sc

In [1843]:
save_json_from_string(response_llm, 'emails_provider_issue.json')

JSON data has been successfully written to 'emails_provider_issue.json'.


True

**Internal Emails Proximus**

**4. BLENDING DATA IN HTML**

**5. SENDING MAILS TO GOPHISH**